In [1]:
# import the libraries to run the code in this file
import requests
import json
from api_keys import rapidapi_key
import pandas as pd
from pymongo import MongoClient

In [2]:
# api request for ski resorts from rapid api DO NOT PULL THIS MORE THAN 1X daily. you'll need to sign up for the rapidapi id to run this 

ski_url = "https://ski-resorts-and-conditions.p.rapidapi.com/v1/resort/"

headers = {
	"X-RapidAPI-Key": rapidapi_key,
	"X-RapidAPI-Host": "ski-resorts-and-conditions.p.rapidapi.com"
}

# results are 6 pages of json records. to get the next page of results you need to set up params that'll scroll through the page numbers
params = {'page': None}

ski_resort_master_data = []

for i in range(1,7):
    params['page'] = i
    response = requests.get(ski_url, params=params, headers=headers)
    ski_resort_master_data.append(response.json())
    print(f'api response page {i}: {response.json()}')

api response page 1: {'page': 1, 'per_page': 25, 'pre_page': None, 'next_page': 2, 'total': 148, 'total_pages': 6, 'data': [{'slug': '49-degrees-north', 'name': '49 Degrees North', 'country': 'US', 'region': 'WA', 'location': {'latitude': 48.302353, 'longitude': -117.564011}, 'url': 'https://api.skiapi.com/v1/resort/49-degrees-north'}, {'slug': 'alpine-meadows', 'name': 'Alpine Meadows', 'country': 'US', 'region': 'CA', 'location': {'latitude': 39.148201, 'longitude': -120.150902}, 'url': 'https://api.skiapi.com/v1/resort/alpine-meadows'}, {'slug': 'alta', 'name': 'Alta', 'country': 'US', 'region': 'UT', 'location': {'latitude': 40.583828, 'longitude': -111.63726}, 'url': 'https://api.skiapi.com/v1/resort/alta'}, {'slug': 'alyeska', 'name': 'Alyeska', 'country': 'US', 'region': 'AK', 'location': {'latitude': 60.961445, 'longitude': -149.102676}, 'url': 'https://api.skiapi.com/v1/resort/alyeska'}, {'slug': 'angel-fire', 'name': 'Angel Fire', 'country': 'US', 'region': 'NM', 'location': 

In [3]:
# this is the unrefined data from the ski resort api
ski_resort_master_data

[{'page': 1,
  'per_page': 25,
  'pre_page': None,
  'next_page': 2,
  'total': 148,
  'total_pages': 6,
  'data': [{'slug': '49-degrees-north',
    'name': '49 Degrees North',
    'country': 'US',
    'region': 'WA',
    'location': {'latitude': 48.302353, 'longitude': -117.564011},
    'url': 'https://api.skiapi.com/v1/resort/49-degrees-north'},
   {'slug': 'alpine-meadows',
    'name': 'Alpine Meadows',
    'country': 'US',
    'region': 'CA',
    'location': {'latitude': 39.148201, 'longitude': -120.150902},
    'url': 'https://api.skiapi.com/v1/resort/alpine-meadows'},
   {'slug': 'alta',
    'name': 'Alta',
    'country': 'US',
    'region': 'UT',
    'location': {'latitude': 40.583828, 'longitude': -111.63726},
    'url': 'https://api.skiapi.com/v1/resort/alta'},
   {'slug': 'alyeska',
    'name': 'Alyeska',
    'country': 'US',
    'region': 'AK',
    'location': {'latitude': 60.961445, 'longitude': -149.102676},
    'url': 'https://api.skiapi.com/v1/resort/alyeska'},
   {'slug

In [4]:
# remove all keys/values above the 'data' level
resort_data = [item for sublist in [item['data'] for item in ski_resort_master_data] for item in sublist]

In [5]:
# show the refined resort master data
resort_data

[{'slug': '49-degrees-north',
  'name': '49 Degrees North',
  'country': 'US',
  'region': 'WA',
  'location': {'latitude': 48.302353, 'longitude': -117.564011},
  'url': 'https://api.skiapi.com/v1/resort/49-degrees-north'},
 {'slug': 'alpine-meadows',
  'name': 'Alpine Meadows',
  'country': 'US',
  'region': 'CA',
  'location': {'latitude': 39.148201, 'longitude': -120.150902},
  'url': 'https://api.skiapi.com/v1/resort/alpine-meadows'},
 {'slug': 'alta',
  'name': 'Alta',
  'country': 'US',
  'region': 'UT',
  'location': {'latitude': 40.583828, 'longitude': -111.63726},
  'url': 'https://api.skiapi.com/v1/resort/alta'},
 {'slug': 'alyeska',
  'name': 'Alyeska',
  'country': 'US',
  'region': 'AK',
  'location': {'latitude': 60.961445, 'longitude': -149.102676},
  'url': 'https://api.skiapi.com/v1/resort/alyeska'},
 {'slug': 'angel-fire',
  'name': 'Angel Fire',
  'country': 'US',
  'region': 'NM',
  'location': {'latitude': 36.387402, 'longitude': -105.273903},
  'url': 'https://ap

In [6]:
# we want our map to contain a layer that will filter national resort passes (Epic and Ikon). To enable this, we'll need a new key named 'pass type'. Add this to all records with a null value
new_key = 'pass_type'
new_key_value = None

for resort in resort_data:
    resort[new_key] = new_key_value

In [7]:
# add the 2023/2024 pass type to the select resorts, epic
for resort in resort_data:
    if resort['name'] in ['Attitash', 'Beaver Creek', 'Breckenridge', 'Crested Butte', 'Heavenly', 'Hunter Mountain', 'Keystone', 'Kirkwood', 'Mount Sunapee', 'Mount Snow', 'Mt Brighton', 'Northstar', 'Okemo', 'Park City', 'Stevens Pass', 'Stowe', 'Telluride', 'Vail', 'Whistler Blackcomb', 'Wildcat']:
        resort['pass type'] = 'Epic'

In [8]:
# add the 2023/2024 pass type to the select resorts, ikon
for resort in resort_data:
    if resort['name'] in ['Alta', 'Alyeska', 'Arapaho Basin', 'Aspen Highlands', 'Aspen Mountain', 'Big Sky', 'Blue Mountain', 'Brighton', 'Buttermilk', 'Camelback Mountain', 'Copper Mountain', 'Crystal Mountain', 'Deer Valley', 'Jackson Hole', 'June Mountain', 'Killington', 'Loon Mountain', 'Mammoth Mountain', 'Mount Bachelor', 'Niseko', 'Palisades Tahoe', 'Pico Mountain', 'Red Mountain', 'Schweitzer', 'Snowbasin', 'Snowbird', 'Snowmass', 'Snowshoe Mountain', 'Solitude', 'Steamboat', 'Stratton', 'Sugarbush', 'Sugarloaf', 'Sun Peaks', 'Sunday River', 'Sun Valley', 'Taos Ski Valley', 'Mont Tremblant', 'Windham Mountain', 'Winter Park']:
        resort['pass type'] = 'Ikon'

In [9]:
# just for review, convert the list of dictionaries to a dataframe to help see what the data would look like if we placed it into a relational db
resort_data_df = pd.DataFrame(resort_data)
resort_data_df

,slug,name,country,region,location,url,pass_type,pass type
0,49-degrees-north,49 Degrees North,US,WA,"{'latitude': 48.302353, 'longitude': -117.564011}",https://api.skiapi.com/v1/resort/49-degrees-north,None,NaN
1,alpine-meadows,Alpine Meadows,US,CA,"{'latitude': 39.148201, 'longitude': -120.150902}",https://api.skiapi.com/v1/resort/alpine-meadows,None,NaN
2,alta,Alta,US,UT,"{'latitude': 40.583828, 'longitude': -111.63726}",https://api.skiapi.com/v1/resort/alta,None,Ikon
3,alyeska,Alyeska,US,AK,"{'latitude': 60.961445, 'longitude': -149.102676}",https://api.skiapi.com/v1/resort/alyeska,None,Ikon
4,angel-fire,Angel Fire,US,NM,"{'latitude': 36.387402, 'longitude': -105.273903}",https://api.skiapi.com/v1/resort/angel-fire,None,NaN
...,...,...,...,...,...,...,...,...
143,whiteface,Whiteface,US,NY,"{'latitude': 44.356506, 'longitude': -73.870613}",https://api.skiapi.com/v1/resort/whiteface,None,NaN
144,wildcat,Wildcat,US,NH,"{'latitude': 44.264626, 'longitude': -71.236725}",https://api.skiapi.com/v1/resort/wildcat,None,Epic
145,windham,Windham Mountain,US,NY,"{'latitude': 42.295597, 'longitude': -74.259941}",https://api.skiapi.com/v1/resort/windham,None,Ikon
146,winter-park,Winter Park,US,CO,"{'latitude': 39.891563, 'longitude': -105.764267}",https://api.skiapi.com/v1/resort/winter-park,None,Ikon


In [10]:
# more review - separate the location into two columns for lat and lon
resort_data_df[['latitude', 'longitude']] = resort_data_df['location'].apply(lambda loc: pd.Series([loc['latitude'], loc['longitude']]))
resort_data_df = resort_data_df.drop('location', axis=1)
resort_data_df

,slug,name,country,region,url,pass_type,pass type,latitude,longitude
0,49-degrees-north,49 Degrees North,US,WA,https://api.skiapi.com/v1/resort/49-degrees-north,None,NaN,48.302353,-117.564011
1,alpine-meadows,Alpine Meadows,US,CA,https://api.skiapi.com/v1/resort/alpine-meadows,None,NaN,39.148201,-120.150902
2,alta,Alta,US,UT,https://api.skiapi.com/v1/resort/alta,None,Ikon,40.583828,-111.637260
3,alyeska,Alyeska,US,AK,https://api.skiapi.com/v1/resort/alyeska,None,Ikon,60.961445,-149.102676
4,angel-fire,Angel Fire,US,NM,https://api.skiapi.com/v1/resort/angel-fire,None,NaN,36.387402,-105.273903
...,...,...,...,...,...,...,...,...,...
143,whiteface,Whiteface,US,NY,https://api.skiapi.com/v1/resort/whiteface,None,NaN,44.356506,-73.870613
144,wildcat,Wildcat,US,NH,https://api.skiapi.com/v1/resort/wildcat,None,Epic,44.264626,-71.236725
145,windham,Windham Mountain,US,NY,https://api.skiapi.com/v1/resort/windham,None,Ikon,42.295597,-74.259941
146,winter-park,Winter Park,US,CO,https://api.skiapi.com/v1/resort/winter-park,None,Ikon,39.891563,-105.764267


In [11]:
# change the order of the columns to help display the data in a more logical order
resort_data_df = resort_data_df[['name', 'slug', 'pass type', 'region', 'country', 'latitude', 'longitude', 'url']]
resort_data_df

,name,slug,pass type,region,country,latitude,longitude,url
0,49 Degrees North,49-degrees-north,NaN,WA,US,48.302353,-117.564011,https://api.skiapi.com/v1/resort/49-degrees-north
1,Alpine Meadows,alpine-meadows,NaN,CA,US,39.148201,-120.150902,https://api.skiapi.com/v1/resort/alpine-meadows
2,Alta,alta,Ikon,UT,US,40.583828,-111.637260,https://api.skiapi.com/v1/resort/alta
3,Alyeska,alyeska,Ikon,AK,US,60.961445,-149.102676,https://api.skiapi.com/v1/resort/alyeska
4,Angel Fire,angel-fire,NaN,NM,US,36.387402,-105.273903,https://api.skiapi.com/v1/resort/angel-fire
...,...,...,...,...,...,...,...,...
143,Whiteface,whiteface,NaN,NY,US,44.356506,-73.870613,https://api.skiapi.com/v1/resort/whiteface
144,Wildcat,wildcat,Epic,NH,US,44.264626,-71.236725,https://api.skiapi.com/v1/resort/wildcat
145,Windham Mountain,windham,Ikon,NY,US,42.295597,-74.259941,https://api.skiapi.com/v1/resort/windham
146,Winter Park,winter-park,Ikon,CO,US,39.891563,-105.764267,https://api.skiapi.com/v1/resort/winter-park


## MongoDB Import
From here, you'll want to connect to your MongoDB server and import the list of dictionaries into the db. 

### Instructions
#### Mac Users:
- Install the Xcode command-line tools with the following command: xcode-select --install
- Install the 6.0 Community Edition with the following commands:
* brew tap mongodb/brew
* brew update
* brew install mongodb-community@6.0

Once you have installed MongoDB, start the server and database with the following commands: 
- Open a terminal window and run:
* brew services start mongodb-community@6.0
* mongosh

To stop the database and server run the following commands:
- In your terminal window: 
* press: Ctrl+C twice
- To stop the MongoDB server, exit the MongoDB shell and run:
* brew services stop mongodb-community@6.0

In [12]:
# connect to MongoDB
client = MongoClient('mongodb://localhost:27017')

# specify the database and collection
resort_master_db = client['ski-tracker-master-db']
collection = resort_master_db['resort_master']

# convert the dataframe to a dictionary and insert it into mongodb
resort_db_records = resort_data
collection.insert_many(resort_db_records)

client.close()